In [20]:
import pandas as pd

#Read Pickle file
import pickle

#AI to lementize
import spacy

#This module implements specialized container datatypes providing alternatives to Python’s general purpose built-in containers, dict, list, set, and tuple.
from collections import Counter
import numpy as np; np.random.seed(0)

#plotting 
import seaborn as sns
import matplotlib.pyplot as plt

#read data from online sources
import pandas_datareader as pdr

#Python package to pretty-print 
import json

# import our libraries
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
import datetime
import calendar

import csv
import requests

from sec_api import QueryApi, RenderApi
from pathlib import Path
import multiprocessing

import os
from pathlib import Path

from sec_api import MappingApi
from datetime import date, timedelta



In [21]:
# Read downloads
FFC1 = pd.read_excel('../Data\Excel xlsx and csv\FFC1 03 to 23.xlsx')
FFC1['FFC1']=FFC1['Close']
FFC1['F1Open']=FFC1['Open']
FFC1=FFC1[['Exchange Date','FFC1','F1Open']]


FFC2 = pd.read_excel('../Data\Excel xlsx and csv\FFC2 03 to 23.xlsx')
FFC2['FFC2']=FFC2['Close']
FFC2['F2Open']=FFC2['Open']
FFC2=FFC2[['Exchange Date','FFC2','F2Open']]


FFC3 = pd.read_excel('../Data\Excel xlsx and csv\FFC3 03 to 23.xlsx')
FFC3['FFC3']=FFC3['Close']
FFC3['F3Open']=FFC3['Open']
FFC3=FFC3[['Exchange Date','FFC3','F3Open']]


FFC4 = pd.read_excel('../Data\Excel xlsx and csv\FFC4 03 to 23.xlsx')
FFC4['FFC4']=FFC4['Close']
FFC4['F4Open']=FFC4['Open']
FFC4=FFC4[['Exchange Date','FFC4','F4Open']]


ED2 = pd.read_excel('../Data\Excel xlsx and csv\ED2 03 to 23.xlsx')
ED2['E2']=ED2['Close']
ED2['E2Open']=ED2['Open']
ED2=ED2[['Exchange Date','E2','E2Open']]

ED3 = pd.read_excel('../Data\Excel xlsx and csv\ED3 03 to 23.xlsx')
ED3['E3']=ED3['Close']
ED3['E3Open']=ED3['Open']
ED3=ED3[['Exchange Date','E3','E3Open']]


ED4 = pd.read_excel('../Data\Excel xlsx and csv\ED4 03 to 23.xlsx')
ED4['E4']=ED4['Close']
ED4['E4Open']=ED4['Open']
ED4=ED4[['Exchange Date','E4','E4Open']]


FOMC = pd.read_excel('../Data\Excel xlsx and csv\FOMC Dates2.xlsx')
FOMC.rename(columns={'Date(UK)': 'FOMC'}, inplace=True)
FOMC=FOMC[['FOMC']]

NFOMC = pd.read_excel('../Data\Excel xlsx and csv\FFC1 03 to 23.xlsx')
NFOMC=NFOMC[['Next','Exchange Date']]



In [22]:
def get_num_days(row):
    year = int(row['Year'])
    month = int(row['Month'])
    return calendar.monthrange(year, month)[1]

In [23]:
NFOMC['Month']= NFOMC['Next'].dt.strftime('%m')
NFOMC['Year']= NFOMC['Next'].dt.strftime('%Y')
NFOMC['D2'] = NFOMC.apply(get_num_days, axis=1)
NFOMC['d2']= NFOMC['Next'].dt.strftime('%d')
NFOMC['d2'] = NFOMC['d2'].astype(int)
NFOMC=NFOMC[['Exchange Date','D2','d2']]
#NFOMC['Date']=pd.to_datetime(NFOMC)

sunday_mask = NFOMC['Exchange Date'].dt.weekday == 6
NFOMC.loc[sunday_mask, 'Exchange Date'] += pd.to_timedelta('1 day')

sunday_mask = FOMC['FOMC'].dt.weekday == 6
FOMC.loc[sunday_mask, 'FOMC'] += pd.to_timedelta('1 day')

In [24]:
Matrix = pd.merge(FFC1, FFC2, on='Exchange Date')
Matrix = pd.merge(Matrix, FFC3, on='Exchange Date')
Matrix = pd.merge(Matrix, FFC4, on='Exchange Date')
Matrix = pd.merge(Matrix, ED2, on='Exchange Date')
Matrix = pd.merge(Matrix, ED3, on='Exchange Date')
Matrix = pd.merge(Matrix, ED4, on='Exchange Date')

Matrix.sort_values(['Exchange Date'], inplace=True)
Matrix.reset_index(inplace=True, drop=True)


Matrix['FF1']=-Matrix['FFC1']+Matrix['F1Open']
Matrix['FF2']=-Matrix['FFC2']+Matrix['F2Open']
Matrix['FF3']=-Matrix['FFC3']+Matrix['F3Open']
Matrix['FF4']=-Matrix['FFC4']+Matrix['F4Open']

Matrix['ED2']=(Matrix['E2']-Matrix['E2Open'])
Matrix['ED3']=(Matrix['E3']-Matrix['E3Open'])
Matrix['ED4']=(Matrix['E4']-Matrix['E4Open'])

Matrix.drop(['FFC1', 'F1Open', 'FFC2', 'F2Open', 'FFC3', 'F3Open',
       'FFC4', 'F4Open', 'E2', 'E2Open', 'E3', 'E3Open', 'E4', 'E4Open',], axis=1, inplace=True)

Matrix

,Exchange Date,FF1,FF2,FF3,FF4,ED2,ED3,ED4
0,2003-04-14,-0.0050,0.010,0.010,0.020,-0.040,-0.060,-0.085
1,2003-04-15,-0.0050,0.000,-0.005,-0.005,0.005,0.030,0.060
2,2003-04-16,-0.0050,0.000,-0.005,-0.015,0.045,0.065,0.075
3,2003-04-17,-0.0050,0.005,0.005,0.005,0.000,-0.025,-0.035
4,2003-04-21,0.0000,0.000,0.000,-0.005,-0.015,0.010,0.010
...,...,...,...,...,...,...,...,...
4963,2022-12-23,0.0025,0.000,0.005,0.010,-0.010,-0.030,-0.045
4964,2022-12-27,0.0000,0.000,0.020,0.020,-0.080,-0.095,-0.095
4965,2022-12-28,0.0000,0.005,-0.005,-0.010,0.020,-0.005,-0.020
4966,2022-12-29,0.0000,-0.005,0.000,-0.005,0.000,-0.005,-0.005


In [25]:
FOMC = pd.read_excel(r'../Data\Excel xlsx and csv\NFOMC.xlsx')
FOMC.rename(columns={'NFOMC': 'Exchange Date'}, inplace=True)
FOMC['Time']= FOMC['Exchange Date'].dt.strftime('%m/%d/%Y')
FOMC['Date']= FOMC['Exchange Date'].dt.strftime('%d')
FOMC['Month']= FOMC['Exchange Date'].dt.strftime('%m')
FOMC['Year']= FOMC['Exchange Date'].dt.strftime('%Y')
FOMC['Month_Year']= FOMC['Exchange Date'].dt.strftime('%m/%Y')
FOMC['Quarter']= FOMC['Exchange Date'].dt.to_period('Q')
FOMC['D2'] = FOMC.apply(get_num_days, axis=1)
FOMC['d2'] = FOMC['Date'].astype(int)
FOMC.rename(columns={'Exchange Date': 'NFOMC'}, inplace=True)
FOMC.rename(columns={'Date(UK)': 'Exchange Date'}, inplace=True)

FOMC['Time']= FOMC['Exchange Date'].dt.strftime('%m/%d/%Y')
FOMC['Date']= FOMC['Exchange Date'].dt.strftime('%d')
FOMC['Month']= FOMC['Exchange Date'].dt.strftime('%m')
FOMC['Year']= FOMC['Exchange Date'].dt.strftime('%Y')
FOMC['Month_Year']= FOMC['Exchange Date'].dt.strftime('%m/%Y')
FOMC['Quarter']= FOMC['Exchange Date'].dt.to_period('Q')
FOMC['D1'] = FOMC.apply(get_num_days, axis=1)
FOMC['d1'] = FOMC['Date'].astype(int)
FOMC['D1_d1'] = FOMC['D1']-FOMC['d1']
FOMC['D1/(D1_d1)'] =  FOMC['D1']/FOMC['D1_d1']

FOMC['D2_d2'] = FOMC['D2']-FOMC['d2']
FOMC['d2/D2'] = FOMC['d2']/FOMC['D2']
FOMC['D2/(D2_d2)'] =  FOMC['D2']/FOMC['D2_d2']

FOMC

,Exchange Date,NFOMC,Time,Date,Month,Year,Month_Year,Quarter,D2,d2,D1,d1,D1_d1,D1/(D1_d1),D2_d2,d2/D2,D2/(D2_d2)
0,2003-05-06,2003-08-12,05/06/2003,06,05,2003,05/2003,2003Q2,31,12,31,6,25,1.240000,19,0.387097,1.631579
1,2003-06-25,2003-09-16,06/25/2003,25,06,2003,06/2003,2003Q2,30,16,30,25,5,6.000000,14,0.533333,2.142857
2,2003-08-12,2003-12-09,08/12/2003,12,08,2003,08/2003,2003Q3,31,9,31,12,19,1.631579,22,0.290323,1.409091
3,2003-09-16,2003-12-09,09/16/2003,16,09,2003,09/2003,2003Q3,31,9,30,16,14,2.142857,22,0.290323,1.409091
4,2003-10-28,2004-01-28,10/28/2003,28,10,2003,10/2003,2003Q4,31,28,31,28,3,10.333333,3,0.903226,10.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,2022-07-27,2022-11-02,07/27/2022,27,07,2022,07/2022,2022Q3,30,2,31,27,4,7.750000,28,0.066667,1.071429
164,2022-09-21,2022-12-14,09/21/2022,21,09,2022,09/2022,2022Q3,31,14,30,21,9,3.333333,17,0.451613,1.823529
165,2022-11-02,2023-02-01,11/02/2022,02,11,2022,11/2022,2022Q4,28,1,30,2,28,1.071429,27,0.035714,1.037037
166,2022-12-14,2023-03-22,12/14/2022,14,12,2022,12/2022,2022Q4,31,22,31,14,17,1.823529,9,0.709677,3.444444


In [26]:
Mat = pd.merge_asof(FOMC, Matrix, on='Exchange Date', direction='nearest')

Mat['MP1']=(Mat['FF1'])*Mat['D1/(D1_d1)']
Mat['MP2']=(Mat['FF2'])
Mat['MP1']= Mat.apply(lambda row: row['MP2'] if row['D1_d1'] <= 7 else row['MP1'], axis=1)

Mat['MP3']=((Mat['FF3'])-(Mat['d2/D2']*Mat['MP1']))*Mat['D2/(D2_d2)']
Mat['MP4']=(Mat['FF4'])
Mat['MP3']= Mat.apply(lambda row: row['MP4'] if row['D2_d2'] <= 7 else row['MP3'], axis=1)

Mat.drop(['NFOMC','Month_Year','Quarter', 'D2', 'd2','D1_d1',  'D1','D2_d2','D1/(D1_d1)', 'd2/D2', 'D2/(D2_d2)','d1',], axis=1, inplace=True)
Mat.rename(columns={'Exchange Date': 'FOMC','Date':'day', 'Month':'month', 'Year':'year'},inplace=True)
Mat[Mat['FOMC']=='2020-03-15']


,FOMC,Time,day,month,year,FF1,FF2,FF3,FF4,ED2,ED3,ED4,MP1,MP2,MP3,MP4
144,2020-03-15,03/15/2020,15,03,2020,0.045,0.035,0.04,0.01,-0.045,-0.055,-0.065,0.087188,0.035,0.016406,0.01


In [27]:
Mat.to_excel('../Data\Excel xlsx and csv\GSSRaw 01 05.xlsx', index=False)
Mat.tail(50)

,FOMC,Time,day,month,year,FF1,FF2,FF3,FF4,ED2,ED3,ED4,MP1,MP2,MP3,MP4
118,2017-03-15,03/15/2017,15,03,2017,0.0025,0.0050,0.000,-0.010,0.030,0.055,0.065,0.004844,0.0050,-0.004238,-0.010
119,2017-05-03,05/03/2017,03,05,2017,-0.0025,0.0000,0.010,0.010,-0.015,-0.015,-0.015,-0.002768,0.0000,0.010000,0.010
120,2017-06-14,06/14/2017,14,06,2017,0.0050,0.0050,0.000,-0.005,0.030,0.030,0.045,0.009375,0.0050,-0.018750,-0.005
121,2017-07-26,07/26/2017,26,07,2017,0.0000,0.0000,-0.005,-0.015,0.035,0.045,0.045,0.000000,0.0000,-0.005172,-0.015
122,2017-09-20,09/20/2017,20,09,2017,0.0000,-0.0050,-0.005,0.015,-0.025,-0.040,-0.050,0.000000,-0.0050,-0.008611,0.015
123,2017-11-01,11/01/2017,01,11,2017,0.0000,-0.0050,-0.005,0.000,-0.010,-0.015,-0.015,0.000000,-0.0050,0.000000,0.000
124,2017-12-13,12/13/2017,13,12,2017,0.0025,0.0050,0.005,0.000,0.020,0.030,0.040,0.004306,0.0050,0.006458,0.000
125,2018-01-31,01/31/2018,31,01,2018,-0.0025,-0.0050,-0.005,0.005,-0.015,-0.015,-0.020,-0.005000,-0.0050,-0.005000,0.005
126,2018-03-21,03/21/2018,21,03,2018,0.0025,0.0050,0.005,-0.005,0.010,0.015,0.005,0.007750,0.0050,0.002897,-0.005
127,2018-05-02,05/02/2018,02,05,2018,0.0000,0.0000,0.000,0.005,-0.005,-0.005,0.005,0.000000,0.0000,0.000000,0.005
